In [1]:
import requests
import pandas as pd

import json

In [105]:
url = 'https://apisidra.ibge.gov.br/values/t/7752/n1/all/v/864/p/last%201/c1264/allxt'
response = requests.get(url)
data = response.json()
pia_produto_rlv = pd.DataFrame(data, columns=data[0])


In [106]:
pia_produto_rlv.columns = pia_produto_rlv.iloc[0]
pia_produto_rlv = pia_produto_rlv[1:]

In [107]:
#Extrair apenas registros cuja coluna 'Classes das atividades industriais e produtos - Prodlist 2016, Prodlist 2019 e Prodlist 2022' que iniciam com o padrão \d{4}\.\d{2}
pia_produto_rlv = pia_produto_rlv[pia_produto_rlv['Classes das atividades industriais e produtos - Prodlist 2016, Prodlist 2019 e Prodlist 2022'].str.match(r'^\d{4}\.\d{2}')]

In [108]:
pia_produto_rlv = pia_produto_rlv[['Ano','Classes das atividades industriais e produtos - Prodlist 2016, Prodlist 2019 e Prodlist 2022','Valor']]
pia_produto_rlv['cod_prodlist'] = pia_produto_rlv['Classes das atividades industriais e produtos - Prodlist 2016, Prodlist 2019 e Prodlist 2022'].str[0:9]

In [97]:
mapeamento = pd.concat([pd.read_excel('./dados/mapeamento/PRODLIST Ind 2022 x NCM 2022.xlsx',names=['prodlist','ncm']),pd.read_excel('./dados/mapeamento/PRODLIST Ind 2019 x NCM 2019.xlsx',names=['prodlist','ncm']),pd.read_excel('./dados/mapeamento/PRODLIST Ind 2016 x NCM 2016.xlsx',names=['prodlist','ncm'])], axis=0,ignore_index=True)


In [43]:
import re
#inserir um 0 à esquerda caso o padrão da coluna 'PRODLIST-Ind 2019' não seja \d{4}\.
mapeamento['prodlist'] = mapeamento['prodlist'].apply(lambda x: f"0{x}" if not re.match(r'\d{4}\.', str(x)) else x).apply(lambda x: f"{x}0" if not re.match(r'\d{4}\.\d{4}', str(x)) else x).apply(lambda x: f"{x}0" if not re.match(r'\d{4}\.\d{4}', str(x)) else x)


In [44]:
mapeamento.shape

(35871, 2)

In [46]:
mapeamento.drop_duplicates().shape

(16351, 2)

In [47]:
mapeamento= mapeamento.drop_duplicates()

In [49]:
mapeamento['cod_ncm'] = mapeamento['ncm'].str.replace('.','', regex=False)

In [53]:
#listar cod_ncm que não possuem 8 dígitos
mapeamento['cod_ncm'].str.len().value_counts()
mapeamento = mapeamento[mapeamento['cod_ncm'].str.len() == 8]

In [54]:
mapeamento['cod_ncm'].str.len().value_counts()


cod_ncm
8    16349
Name: count, dtype: int64

In [69]:
exportacoes_mundo_2023 = pd.read_excel('./dados/comex/Exportação  Mundo 2023 NCM.xlsx')
exportacoes_usa_2023 = pd.read_excel('./dados/comex/Exportação USA 2023 NCM.xlsx.xlsx')


In [70]:
exportacoes_mundo_2023['cod_ncm'] = exportacoes_mundo_2023['Código NCM'].astype(str).str.zfill(8)
exportacoes_usa_2023['cod_ncm'] = exportacoes_usa_2023['Código NCM'].astype(str).str.zfill(8)

In [71]:
exportacoes_mundo_2023.head(2)

,Código NCM,Descrição NCM,2023 - Valor US$ FOB,cod_ncm
0,12019000,"Soja, mesmo triturada, exceto para semeadura",53238560791,12019000
1,27090010,Óleos brutos de petróleo,42611078672,27090010


In [72]:
mapeamento.head(2)

,prodlist,ncm,cod_ncm
0,0500.2010,2701.19.00,27011900
1,0500.2030,2701.11.00,27011100


In [73]:
exportacoes_mundo_2023 = exportacoes_mundo_2023.merge(mapeamento.drop('ncm',axis=1), on='cod_ncm', how='left')
exportacoes_usa_2023 = exportacoes_usa_2023.merge(mapeamento.drop('ncm',axis=1), on='cod_ncm', how='left')

In [74]:
exportacoes_mundo_2023.head()

,Código NCM,Descrição NCM,2023 - Valor US$ FOB,cod_ncm,prodlist
0,12019000,"Soja, mesmo triturada, exceto para semeadura",53238560791,12019000,NaN
1,27090010,Óleos brutos de petróleo,42611078672,27090010,0600.2030
2,26011100,"Minérios de ferro e seus concentrados, exceto ...",27298053224,26011100,0710.2015
3,26011100,"Minérios de ferro e seus concentrados, exceto ...",27298053224,26011100,0710.2030
4,10059010,"Milho em grão, exceto para semeadura",13459787656,10059010,NaN


In [75]:
ncms_sem_prodlist_mundo = exportacoes_mundo_2023[exportacoes_mundo_2023['prodlist'].isna()]['2023 - Valor US$ FOB'].sum()/ exportacoes_mundo_2023['2023 - Valor US$ FOB'].sum()
ncm_sem_prodlist_usa = exportacoes_usa_2023[exportacoes_usa_2023['prodlist'].isna()]['2023 - Valor US$ FOB'].sum()/ exportacoes_usa_2023['2023 - Valor US$ FOB'].sum()

print(f'Participação dos NCMs sem Prodlist na exportação mundial: {ncms_sem_prodlist_mundo:.2%}')
print(f'Participação dos NCMs sem Prodlist na exportação para os EUA: {ncm_sem_prodlist_usa:.2%}')

Participação dos NCMs sem Prodlist na exportação mundial: 14.81%
Participação dos NCMs sem Prodlist na exportação para os EUA: 3.12%


In [76]:
#Taxa média de câmbio em 2023 - compra
taxa_cambio = 4.99  # Exemplo de taxa média de câmbio em 2023
exportacoes_mundo_2023['2023 - Valor R$ FOB Mundo'] = exportacoes_mundo_2023['2023 - Valor US$ FOB'] * taxa_cambio
exportacoes_usa_2023['2023 - Valor R$ FOB USA'] = exportacoes_usa_2023['2023 - Valor US$ FOB'] * taxa_cambio

In [77]:
exportacoes_mundo_2023 = exportacoes_mundo_2023.dropna(subset=['prodlist'])
exportacoes_usa_2023 = exportacoes_usa_2023.dropna(subset=['prodlist'])

In [82]:
exportacoes_usa_2023.shape

(7335, 6)

In [81]:
exportacoes_mundo_2023.shape

(11705, 6)

In [83]:
exportacoes_mundo_2023 = exportacoes_mundo_2023.groupby('prodlist').agg({'2023 - Valor R$ FOB Mundo': 'sum'}).reset_index()
exportacoes_usa_2023 = exportacoes_usa_2023.groupby('prodlist').agg({'2023 - Valor R$ FOB USA': 'sum'}).reset_index()



In [85]:
exportacoes = exportacoes_mundo_2023.merge(exportacoes_usa_2023, on='prodlist', how='outer').fillna(0)

In [87]:
verificacao_ía_produto_vazio = pia_produto_rlv.merge(exportacoes, left_on='cod_prodlist', right_on='prodlist', how='right')
verificacao_ía_produto_vazio.query('Valor.isna()')

,Ano,"Classes das atividades industriais e produtos - Prodlist 2016, Prodlist 2019 e Prodlist 2022",Valor,cod_prodlist,prodlist,2023 - Valor R$ FOB Mundo,2023 - Valor R$ FOB USA
0,NaN,NaN,NaN,NaN,1011.2045,2.484606e+09,7.014848e+08
1,NaN,NaN,NaN,NaN,1011.2055,3.313964e+08,1.035435e+08
2,NaN,NaN,NaN,NaN,1011.2065,1.405934e+09,1.231338e+09
3,NaN,NaN,NaN,NaN,1011.2095,2.213978e+06,0.000000e+00
4,NaN,NaN,NaN,NaN,1013.2045,8.219026e+08,8.532209e+07
...,...,...,...,...,...,...,...
3320,NaN,NaN,NaN,NaN,3511.2010,2.766594e+09,0.000000e+00
3321,NaN,NaN,NaN,NaN,3520.2010,1.317360e+04,6.187600e+03
3322,NaN,NaN,NaN,NaN,3520.2020,6.906160e+03,0.000000e+00
3323,NaN,NaN,NaN,NaN,3520.2030,4.246490e+03,0.000000e+00


In [109]:
pia_produto_rlv.head(2)

,Ano,"Classes das atividades industriais e produtos - Prodlist 2016, Prodlist 2019 e Prodlist 2022",Valor,cod_prodlist
2,2023,0500.2010 Carvão mineral e outros combustíveis...,1621736,0500.2010
3,2023,"0500.2030 Hulhas, mesmo em pó, finos de carvão...",126178,0500.2030


In [110]:
pia_produto_rlv_bck = pia_produto_rlv.copy()

In [129]:
pia_produto_rlv = pia_produto_rlv_bck

In [130]:
#dropar linhas em que o valor da coluna 'Valor' não pode ser convertido para float
pia_produto_rlv = pia_produto_rlv[pia_produto_rlv['Valor']!='-']
pia_produto_rlv = pia_produto_rlv[pia_produto_rlv['Valor']!='X']
pia_produto_rlv['Valor'] = pia_produto_rlv['Valor'].astype(float)
pia_produto_rlv['Valor'] =pia_produto_rlv['Valor']*1000

In [131]:
print(f'o valor de x é {1.621736e+09}')

o valor de x é 1621736000.0


In [132]:
pia_produto_rlv.merge(exportacoes, left_on='cod_prodlist', right_on='prodlist', how='left').fillna(0)

,Ano,"Classes das atividades industriais e produtos - Prodlist 2016, Prodlist 2019 e Prodlist 2022",Valor,cod_prodlist,prodlist,2023 - Valor R$ FOB Mundo,2023 - Valor R$ FOB USA
0,2023,0500.2010 Carvão mineral e outros combustíveis...,1.621736e+09,0500.2010,0,0.000000e+00,0.00
1,2023,"0500.2030 Hulhas, mesmo em pó, finos de carvão...",1.261780e+08,0500.2030,0500.2030,1.776615e+06,0.00
2,2023,0500.9010 Serviço de beneficiamento de carvão ...,1.494900e+07,0500.9010,0,0.000000e+00,0.00
3,2023,"0600.2010 Gás natural, liquefeito ou no estado...",2.614563e+10,0600.2010,0600.2010,2.280291e+08,4496948.08
4,2023,0600.2020 Óleos brutos de minerais betuminosos,4.706370e+08,0600.2020,0600.2020,3.489258e+05,1701.59
...,...,...,...,...,...,...,...
2676,2023,3321.9140 Serviços de instalação e montagem de...,7.895100e+07,3321.9140,0,0.000000e+00,0.00
2677,2023,3321.9160 Serviços de instalação e montagem de...,2.020000e+07,3321.9160,0,0.000000e+00,0.00
2678,2023,3321.9170 Serviços de instalação e montagem de...,7.995700e+08,3321.9170,0,0.000000e+00,0.00
2679,2023,3329.9010 Serviços de montagem e acabamento de...,3.181430e+08,3329.9010,0,0.000000e+00,0.00


In [133]:
pia_produto_rlv = pia_produto_rlv.merge(exportacoes, left_on='cod_prodlist', right_on='prodlist', how='left').fillna(0)

In [134]:
pia_produto_rlv['participacao_exportacao_usa'] = pia_produto_rlv['2023 - Valor R$ FOB USA']/pia_produto_rlv['Valor']
pia_produto_rlv['participacao_exportacao_mundo'] = pia_produto_rlv['2023 - Valor R$ FOB Mundo']/pia_produto_rlv['Valor']

In [135]:
pia_produto_rlv

,Ano,"Classes das atividades industriais e produtos - Prodlist 2016, Prodlist 2019 e Prodlist 2022",Valor,cod_prodlist,prodlist,2023 - Valor R$ FOB Mundo,2023 - Valor R$ FOB USA,participacao_exportacao_usa,participacao_exportacao_mundo
0,2023,0500.2010 Carvão mineral e outros combustíveis...,1.621736e+09,0500.2010,0,0.000000e+00,0.00,0.000000,0.000000
1,2023,"0500.2030 Hulhas, mesmo em pó, finos de carvão...",1.261780e+08,0500.2030,0500.2030,1.776615e+06,0.00,0.000000,0.014080
2,2023,0500.9010 Serviço de beneficiamento de carvão ...,1.494900e+07,0500.9010,0,0.000000e+00,0.00,0.000000,0.000000
3,2023,"0600.2010 Gás natural, liquefeito ou no estado...",2.614563e+10,0600.2010,0600.2010,2.280291e+08,4496948.08,0.000172,0.008722
4,2023,0600.2020 Óleos brutos de minerais betuminosos,4.706370e+08,0600.2020,0600.2020,3.489258e+05,1701.59,0.000004,0.000741
...,...,...,...,...,...,...,...,...,...
2676,2023,3321.9140 Serviços de instalação e montagem de...,7.895100e+07,3321.9140,0,0.000000e+00,0.00,0.000000,0.000000
2677,2023,3321.9160 Serviços de instalação e montagem de...,2.020000e+07,3321.9160,0,0.000000e+00,0.00,0.000000,0.000000
2678,2023,3321.9170 Serviços de instalação e montagem de...,7.995700e+08,3321.9170,0,0.000000e+00,0.00,0.000000,0.000000
2679,2023,3329.9010 Serviços de montagem e acabamento de...,3.181430e+08,3329.9010,0,0.000000e+00,0.00,0.000000,0.000000


In [136]:
pia_produto_rlv.to_excel('./dados/participacao_exportacao_americana_pia.xlsx', index=False)

In [137]:
mapeamento.to_excel('./dados/mapeamento/prodlist_ncm_consolidada.xlsx', index=False)

In [220]:
url_pia_empresa = 'https://apisidra.ibge.gov.br/values/t/1842/n1/all/v/824,825/p/last%201/c12762/all'

data = requests.get(url_pia_empresa)
data = data.json()
pia_empresa_rlv = pd.DataFrame(data, columns=data[0])


In [221]:
pia_empresa_rlv.columns = pia_empresa_rlv.iloc[0]
pia_empresa_rlv = pia_empresa_rlv[1:]


In [222]:
pia_empresa_rlv.head(2)


,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Brasil (Código),Brasil,Variável (Código),Variável,Ano (Código),Ano,Classificação Nacional de Atividades Econômicas (CNAE 2.0) (Código),Classificação Nacional de Atividades Econômicas (CNAE 2.0)
1,1,Brasil,40,Mil Reais,6353568639,1,Brasil,824,Total da receita líquida de vendas,2023,2023,117897,Total
2,1,Brasil,40,Mil Reais,456258347,1,Brasil,824,Total da receita líquida de vendas,2023,2023,116880,B Indústrias extrativas


In [223]:
##Filtrar apenas registros cujos valores da coluna 'Classificação Nacional de Atividades Econômicas (CNAE 2.0)' siga o padrão \d{2}\.\d{1} 
pia_empresa_rlv = pia_empresa_rlv[pia_empresa_rlv['Classificação Nacional de Atividades Econômicas (CNAE 2.0)'].str.match(r'^\d{2}\.\d{1}')]
pia_empresa_rlv['cnae'] = pia_empresa_rlv['Classificação Nacional de Atividades Econômicas (CNAE 2.0)'].str[0:4].str.replace('.','', regex=False)
pia_empresa_rlv['cnae_descricao'] = pia_empresa_rlv['Classificação Nacional de Atividades Econômicas (CNAE 2.0)'].str[5:]

In [224]:
pia_empresa_rlv = pia_empresa_rlv[['Ano','Variável','cnae','cnae_descricao','Valor']]

In [225]:
pia_empresa_rlv.head()

,Ano,Variável,cnae,cnae_descricao,Valor
4,2023,Total da receita líquida de vendas,050,Extração de carvão mineral,1941673
6,2023,Total da receita líquida de vendas,060,Extração de petróleo e gás natural,163281926
8,2023,Total da receita líquida de vendas,071,Extração de minério de ferro,196958255
9,2023,Total da receita líquida de vendas,072,Extração de minerais metálicos não-ferrosos,27466066
11,2023,Total da receita líquida de vendas,081,"Extração de pedra, areia e argila",25005569


In [226]:
#Linhas em que o valor da coluna 'Valor' não pode ser convertido para float, trocar por -1
pia_empresa_rlv['Valor'] = pd.to_numeric(pia_empresa_rlv['Valor'], errors='coerce').fillna(-1)
pia_empresa_rlv['Valor'] = pia_empresa_rlv['Valor']*1000

In [227]:
#Fazer o pivot transformando a coluna 'Variável' em colunas e preenchendo com os valores da coluna 'Valor'
pia_empresa_rlv = pia_empresa_rlv.pivot_table(index=['Ano','cnae','cnae_descricao'], columns='Variável', values='Valor')

In [228]:
pia_empresa_rlv.reset_index(inplace=True)

In [229]:
pia_empresa_rlv.head(2)

Variável,Ano,cnae,cnae_descricao,Receita líquida de vendas de produtos e serviços industriais,Total da receita líquida de vendas
0,2023,050,Extração de carvão mineral,1.850253e+09,1.941673e+09
1,2023,060,Extração de petróleo e gás natural,1.626363e+11,1.632819e+11


In [239]:
#fazer um merge parcial, com o cnae de pia_empresa_rlv e os três primeiros caracteres de pia_produto_rlv
pia_merge = pd.merge(pia_empresa_rlv.drop('Ano',axis=1), pia_produto_rlv, left_on='cnae', right_on=pia_produto_rlv['cod_prodlist'].str[:3], how='outer',indicator=True)

In [240]:
pia_merge.shape

(2684, 14)

In [241]:
pia_merge.query('_merge == "left_only"').shape

(3, 14)

In [242]:
pia_merge.query('_merge == "right_only"').shape

(0, 14)

In [243]:
#substituir valores negativos de Receita líquida de vendas de produtos e serviços industriais ou Total da receita líquida de vendas por 0
pia_empresa_rlv['Receita líquida de vendas de produtos e serviços industriais'] = pia_empresa_rlv['Receita líquida de vendas de produtos e serviços industriais'].apply(lambda x: 0 if x < 0 else x)
pia_empresa_rlv['Total da receita líquida de vendas'] = pia_empresa_rlv['Total da receita líquida de vendas'].apply(lambda x: 0 if x < 0 else x)
#substituir valores negativos de Receita líquida de vendas de produtos e serviços industriais ou Total da receita líquida de vendas por 0
pia_merge['Receita líquida de vendas de produtos e serviços industriais'] = pia_merge['Receita líquida de vendas de produtos e serviços industriais'].apply(lambda x: 0 if x < 0 else x)
pia_merge['Total da receita líquida de vendas'] = pia_merge['Total da receita líquida de vendas'].apply(lambda x: 0 if x < 0 else x)


In [246]:
pia_merge.query('_merge == "both"').drop(columns=['_merge']).to_excel('./dados/participacao_exportacao_americana_pia_empresa.xlsx', index=False)

In [210]:
pia_merge.query('_merge == "both"').shape

(2636, 15)

In [215]:
pia_merge['_merge'].value_counts()

_merge
both          2636
right_only      45
left_only        1
Name: count, dtype: int64

In [252]:
pia_merge.head(2)

,cnae,cnae_descricao,Receita líquida de vendas de produtos e serviços industriais,Total da receita líquida de vendas,Ano,"Classes das atividades industriais e produtos - Prodlist 2016, Prodlist 2019 e Prodlist 2022",Valor,cod_prodlist,prodlist,2023 - Valor R$ FOB Mundo,2023 - Valor R$ FOB USA,participacao_exportacao_usa,participacao_exportacao_mundo,_merge
0,050,Extração de carvão mineral,1.850253e+09,1.941673e+09,2023,0500.2010 Carvão mineral e outros combustíveis...,1.621736e+09,0500.2010,0,0.00,0.0,0.0,0.00000,both
1,050,Extração de carvão mineral,1.850253e+09,1.941673e+09,2023,"0500.2030 Hulhas, mesmo em pó, finos de carvão...",1.261780e+08,0500.2030,0500.2030,1776614.65,0.0,0.0,0.01408,both


In [253]:
pia_merge['cnae_3d_prodlist'] = pia_merge['cod_prodlist'].str[:3]


In [255]:
pia_merge.columns

Index(['cnae', 'cnae_descricao',
       'Receita líquida de vendas de produtos e serviços industriais',
       'Total da receita líquida de vendas', 'Ano',
       'Classes das atividades industriais e produtos - Prodlist 2016, Prodlist 2019 e Prodlist 2022',
       'Valor', 'cod_prodlist', 'prodlist', '2023 - Valor R$ FOB Mundo',
       '2023 - Valor R$ FOB USA', 'participacao_exportacao_usa',
       'participacao_exportacao_mundo', '_merge', 'cnae_3d_prodlist'],
      dtype='object')

In [257]:
pia_merge.groupby(['cnae','cnae_descricao','Receita líquida de vendas de produtos e serviços industriais','Total da receita líquida de vendas'])[['2023 - Valor R$ FOB Mundo','2023 - Valor R$ FOB USA']].sum().reset_index().to_excel("./dados/participacao_exportacao_americana_pia_empresa_cnae.xlsx", index=False)

In [261]:
mapeamento['cod_ncm'] = mapeamento['ncm'].str.replace('.','', regex=False)

In [262]:
exportacoes_mundo_2023_semprod = pd.read_excel('./dados/comex/Exportação  Mundo 2023 NCM.xlsx')
exportacoes_usa_2023_sem_prod = pd.read_excel('./dados/comex/Exportação USA 2023 NCM.xlsx.xlsx')

exportacoes_mundo_2023_semprod['cod_ncm'] = exportacoes_mundo_2023_semprod['Código NCM'].astype(str).str.zfill(8)
exportacoes_usa_2023_sem_prod['cod_ncm'] = exportacoes_usa_2023_sem_prod['Código NCM'].astype(str).str.zfill(8)
exportacoes_mundo_2023_semprod = exportacoes_mundo_2023_semprod.merge(mapeamento.drop('ncm',axis=1), on='cod_ncm', how='left')
exportacoes_usa_2023_sem_prod = exportacoes_usa_2023_sem_prod.merge(mapeamento.drop('ncm',axis=1), on='cod_ncm', how='left')



In [270]:
exportacoes_mundo_2023_semprod.drop("Código NCM", axis=1).rename(columns={'2023 - Valor US$ FOB': 'Valor Fob Mundo'}).merge(exportacoes_usa_2023_sem_prod.drop(["Código NCM","Descrição NCM","prodlist"], axis=1).rename(columns={'2023 - Valor US$ FOB': 'Valor Fob EUA'}), on='cod_ncm', how='right').query('prodlist.isna()').drop('prodlist', axis=1).to_excel('./dados/participacao_exportacao_americana_sem_prodlist.xlsx', index=False)